In [ ]:
import pandas as pd
from langdetect import detect, LangDetectException
from multiprocessing import Pool, cpu_count
import matplotlib.pyplot as plt

# Загрузка данных
df = pd.read_csv('final_combined_toxicity_data_updated.csv')

# Случайный выбор 100  строк
df_sample = df.sample(n=100, random_state=42)

# Функция для определения языка
def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return "unknown"

# Параллельная обработка
def parallel_detect_language(texts):
    with Pool(cpu_count()) as pool:
        results = pool.map(detect_language, texts)
    return results

# Применяем функцию к столбцу comment_text (по частям, чтобы не перегружать память)
print("Определение языка для комментариев...")
chunk_size = 10  # Размер части данных для обработки
languages = []
for i in range(0, len(df_sample), chunk_size):
    chunk = df_sample['comment_text'].iloc[i:i + chunk_size]
    languages.extend(parallel_detect_language(chunk))
    print(f"Обработано {i + chunk_size} строк из {len(df_sample)}")

df_sample['language'] = languages

# 1. Распределение языков
language_distribution = df_sample['language'].value_counts()
print("\nРаспределение языков:")
print(language_distribution)

# 2. Топ-10 языков
top_languages = language_distribution.head(10)
print("\nТоп-10 языков:")
print(top_languages)

# 3. Анализ токсичности по языкам
toxic_by_language = df_sample.groupby('language')[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].mean()
print("\nСредние значения меток токсичности по языкам:")
print(toxic_by_language)

# 4. Визуализация распределения языков
plt.figure(figsize=(10, 6))
top_languages.plot(kind='bar', color='skyblue')
plt.title('Топ-10 языков комментариев')
plt.xlabel('Язык')
plt.ylabel('Количество комментариев')
plt.show()

# 5. Визуализация токсичности по языкам
plt.figure(figsize=(10, 6))
toxic_by_language['toxic'].sort_values(ascending=False).head(10).plot(kind='bar', color='orange')
plt.title('Топ-10 языков по доле токсичных комментариев')
plt.xlabel('Язык')
plt.ylabel('Доля токсичных комментариев')
plt.show()

C:\Users\bokar\AppData\Local\Temp\ipykernel_11328\134851830.py:7: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('final_combined_toxicity_data_updated.csv')


Определение языка для комментариев...
